In [1]:
import pandas as pd

def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()

def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

### Buy the position when:

 * weekly RSI(30)  ≥  daily RSI(30)  >  70
 * Close  >  MA(10)  >  MA(20)  >  MA(50)  >  MA(100)

### Close the position when:
 * Daily close is more than 2% below MA(10)
 * 8% fixed stop loss is hit

In [4]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [6]:
import yfinance as yf
GOOGLE=yf.download("GOOG",start="2018-01-01", end="2020-06-30",auto_adjust = True)

from backtesting import Backtest

backtest = Backtest(GOOGLE, System, commission=.002)
stats= backtest.run()

[*********************100%***********************]  1 of 1 completed


In [7]:
%%time

backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))

CPU times: user 202 ms, sys: 50.7 ms, total: 252 ms
Wall time: 6.44 s


Start                     2018-01-02 00:00:00
End                       2020-06-29 00:00:00
Duration                    909 days 00:00:00
Exposure Time [%]                     2.71132
Equity Final [$]                      10167.1
Equity Peak [$]                         10315
Return [%]                            1.67137
Buy & Hold Return [%]                 30.9831
Max. Drawdown [%]                    -3.06834
Avg. Drawdown [%]                    -1.27755
Max. Drawdown Duration      259 days 00:00:00
Avg. Drawdown Duration      140 days 00:00:00
# Trades                                    2
Win Rate [%]                               50
Best Trade [%]                        2.20809
Worst Trade [%]                     -0.235516
Avg. Trade [%]                       0.978895
Max. Trade Duration          24 days 00:00:00
Avg. Trade Duration          12 days 00:00:00
Profit Factor                         9.37555
Expectancy [%]                         1.2218
SQN                               

In [8]:
backtest.plot()

### Result:
##### While the strategy doesn't perform as well as simple buy & hold, it does so with significantly lower exposure (time in market).
##### In conclusion, to test strategies on multiple time frames, you need to pass in OHLC data in the lowest time frame, then resample it to higher time frames, apply the indicators, then resample back to the lower time frame, filling in the in-betweens.

In [7]:
#These are the trades that would executed with this strategy.
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,35,200,207,282.95478,275.00,-278.41730,-0.028113,2005-06-06,2005-06-15,9 days
1,32,215,235,299.49780,295.01,-143.60960,-0.014984,2005-06-27,2005-07-26,29 days
2,25,304,328,372.60372,408.70,902.40700,0.096876,2005-11-01,2005-12-06,35 days
3,22,352,357,472.21254,451.17,-462.93588,-0.044562,2006-01-11,2006-01-19,8 days
